_________
_Notebook créé par Lise Brisset, le 21/12/2024, dans le cadre du cours de d'Apprentissage Artificiel (enseignant : Loïc Grobol)._

Groupe du projet ML2024 DEFT09 : Patricia Augustyn, Lise Brisset et Solomiia Korol.

Ce notebook permet d'extraire les vecteurs du corpus DEFT09.
Il suit l'étape du notebook _extract_train_test.ipynb_ .

___________

In [1]:
# Importation des librairies nécessaires au programme : 
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

## Récupération des données train et test des fichiers JSON :

In [2]:
# Définition de la fonction qui permet de lire le fichier JSON :

def lire_fichier_json(nom_fichier):
    with open(nom_fichier, "r") as fichier:
        file = fichier.read()
        donnees = json.loads(file)
    return donnees

# Définition de la fonction qui permet de récupérer les textes des données JSON :

def recuperer_textes(donnees, type_donnees): 
    textes = []
    for element in donnees[type_donnees]:
        textes.append(element["texte"])
    return textes

# Définition de la fonction qui permet de récupérer les partis politiques des données JSON :

def recuperer_partis(donnees, type_donnees):
    partis = []
    for element in donnees[type_donnees]:
        partis.append(element["parti"])
    return partis

# Définition de la fonction qui permet d'extraire les données du fichier JSON : 

def extraction_donnees(nom_fichier, types_donnees):
    donnees = lire_fichier_json(nom_fichier)
    textes = recuperer_textes(donnees, types_donnees)
    partis = recuperer_partis(donnees, types_donnees)
    print(f"La liste des textes et celle des partis politiques de {types_donnees} ont bien été extraites 😄.")
    return textes, partis

In [3]:
# extraction des données d'entraînement à partir des fichiers JSON :
nom_fichier_train = "./data_parlement/train/deft09_parlement_train.json"
textes_train, partis_train = extraction_donnees(nom_fichier_train, "train")

# extraction des données de test à partir des fichiers JSON :
nom_fichier_test = "./data_parlement/test/deft09_parlement_test.json"
textes_test, partis_test = extraction_donnees(nom_fichier_test, "test")

La liste des textes et celle des partis politiques de train ont bien été extraites 😄.
La liste des textes et celle des partis politiques de test ont bien été extraites 😄.


In [4]:
# Affichage de quelques exemples : 
print("Exemples de textes d'entraînement :")
for i in range(3):
    print(f"Texte {i+1} : {textes_train[i]}")
    print(f"Parti politique : {partis_train[i]}")
    print("\n")

print("Exemples de textes de test :")
for i in range(3):
    print(f"Texte {i+1} : {textes_test[i]}")
    print(f"Parti politique : {partis_test[i]}")
    print("\n")

Exemples de textes d'entraînement :
Texte 1 : Monsieur le Président, j'ai toujours fait preuve d'un certain scepticisme quant aux comportements moralistes et policiers qui encouragent carrément la délation et qui permettent d'éviter des cas de fraude seulement par le biais de la création d'instruments de contrôle toujours plus envahissants par rapport aux personnes. Nous pouvons certainement - mon groupe et moi-même - soutenir avec conviction le rapport du président Napolitano. La modification du règlement et la proposition de décision répondent en effet sérieusement à une préoccupation qui a été soulevée récemment et que M. Dell'Alba a été le dernier à exprimer : ne pas soumettre les députés à des contrôles excessifs et arbitraires. Je ne vois aucune raison pour retarder davantage cette décision. Si un tel abus devait se produire, le Parlement européen et les députés disposent de larges possibilités pour se défendre. Des procédures internes plus transparentes et plus claires rendront 

## Vectorisation des données train et test :

Nous allons vectoriser avec la méthode TF-IDF, car lors de nos travaux sur le TP ML2024 sur le corpus 20 newsgroups, nous avons remarqué que cette méthode de vectorisation est plus adaptée à un problème de classification de texte plutôt que la méthode de comptage de mot simple (_CountVectorizer_). 

In [5]:
# Définition de la fonction qui permet de vectoriser les textes :

def vectoriser_textes(textes_train, textes_test):
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(textes_train)
    X_test = vectorizer.transform(textes_test)
    print("Les textes ont bien été vectorisés 😄.")
    return X_train, X_test, vectorizer

In [6]:
# Vectorisation des textes d'entraînement et de test :
datas_train, datas_test, vectorizer = vectoriser_textes(textes_train, textes_test)

Les textes ont bien été vectorisés 😄.


In [7]:
# Affichage de la taille des données d'entraînement et de test :
print(f"Taille des données d'entraînement : {datas_train.shape}")
print(f"Taille des données de test : {datas_test.shape}")

Taille des données d'entraînement : (19370, 54760)
Taille des données de test : (12917, 54760)


## Conversion des noms de classes en numéro : 

Nous avons cinqs classes différentes, nous allons donc convertir chaque noms de partis politiques en un numéro de classe entre 1 et 5 :
- Verts-ALE = 1, 
- GUE-NGL = 2, 
- PSE = 3, 
- ELDR = 4, 
- PPE-DE = 5

In [8]:
# Définition d'une fonction qui convertie les noms des classes en id :

def convert_classes(liste_classes, dico_equivalences):
    partis_convertis = []
    for i, parti in enumerate(liste_classes):
        try:
            partis_convertis.append(dico_equivalences[parti])
        except KeyError as e:
            print(e, "au document :", i+1)
    print("Convertion terminée !")
    return partis_convertis

In [9]:
# Conversion de la liste de partis politiques ordonnés en une liste équivalente des classes avec des ids allant de 1 à 5 :
dico_equivalences = {"Verts-ALE" : 1, "GUE-NGL" : 2, "PSE" : 3, "ELDR" : 4, "PPE-DE" : 5}

# convertion des classes train et test : 
partis_train_convertis = convert_classes(partis_train, dico_equivalences)
partis_test_convertis = convert_classes(partis_test, dico_equivalences)

Convertion terminée !
'' au document : 1175
'' au document : 4574
Convertion terminée !


Comme il n'y a que deux documents qui n'ont pas de partis attitré, nous allons les retirer nous même. Il n'y en a que deux, de plus après vérification, les texte associé à ce ids sont vides donc nous pouvons totalement les retirer !

Deux document était vide dans le JSON, nous les avosn donc supprimé manuellement. Ce sont les documents avec pour ID : 1175 et 4574 dans les données de test. 

## Enregistrement des données dans un format tabulaire

In [10]:
print(f" 📚 Le corpus test converti 📚 : {partis_test_convertis}")
print(f"📚 Le corpus train converti 📚 : {partis_train_convertis}")

 📚 Le corpus test converti 📚 : [4, 2, 5, 2, 5, 1, 5, 5, 2, 5, 1, 1, 5, 5, 2, 3, 5, 2, 3, 1, 5, 5, 5, 5, 5, 5, 1, 1, 3, 1, 5, 4, 5, 3, 1, 5, 5, 5, 3, 5, 1, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 5, 4, 1, 4, 5, 2, 3, 3, 2, 4, 5, 3, 4, 1, 5, 5, 5, 4, 3, 5, 3, 2, 1, 3, 3, 4, 5, 4, 4, 1, 5, 1, 5, 3, 3, 5, 3, 3, 1, 2, 1, 3, 4, 4, 4, 5, 4, 5, 3, 5, 3, 5, 1, 5, 5, 2, 5, 5, 2, 3, 4, 4, 5, 3, 1, 4, 2, 5, 3, 5, 5, 5, 5, 1, 2, 3, 5, 2, 5, 3, 5, 5, 5, 3, 5, 3, 3, 5, 1, 4, 1, 5, 4, 2, 2, 3, 2, 3, 3, 1, 5, 5, 2, 3, 3, 5, 3, 4, 5, 3, 3, 5, 3, 5, 5, 1, 5, 3, 1, 1, 5, 5, 5, 3, 1, 5, 4, 5, 3, 2, 3, 4, 3, 3, 2, 4, 2, 5, 5, 5, 1, 2, 4, 2, 5, 5, 2, 4, 5, 5, 5, 2, 4, 5, 5, 1, 5, 5, 1, 5, 2, 3, 5, 3, 5, 1, 4, 3, 5, 1, 2, 3, 3, 5, 3, 4, 2, 2, 3, 5, 5, 3, 5, 5, 1, 5, 3, 5, 1, 3, 5, 3, 4, 3, 2, 5, 3, 5, 5, 2, 5, 3, 1, 3, 2, 1, 5, 1, 1, 5, 5, 3, 5, 1, 3, 4, 4, 2, 1, 1, 3, 3, 4, 3, 3, 5, 5, 5, 4, 5, 5, 2, 5, 5, 4, 3, 1, 5, 5, 5, 5, 1, 3, 3, 3, 3, 4, 5, 4, 2, 3, 4, 5, 2, 5, 5, 5, 2, 1, 5, 4, 2, 2, 1, 3, 5, 3, 2, 2, 3, 4, 4,

In [11]:
# Créez un DataFrame pour les données d'entraînement
df_train = pd.DataFrame(partis_train_convertis, columns=["Classe"])
df_train.to_csv("./data_csv/train/partis_train_convertis.csv", index=False)

# Créez un DataFrame pour les données de test
df_test = pd.DataFrame(partis_test_convertis, columns=["Classe"])
df_test.to_csv("./data_csv/test/partis_test_convertis.csv", index=False)


# XXXX JE N'AI PAS LANCÉ CE QUI EST EN BAS XXX

## Enregistrement des vecteurs au format Parquet :

Ajouter des explications de pourquoi avoir choisi le format Parquet.

In [ ]:
# On va convertir la matrice TF-IDF en liste de listes afin d'en faire ensuite un dictionnaire :
features_train = datas_train.toarray().tolist()  # Convertir en liste de listes

# Création du dictionnaire :
dico_data_train = {
    "features": features_train,
    "class": partis_train
}

In [ ]:
# La même chose aves les données de test : 
features_test = datas_test.toarray().tolist()

# création du dictionnaire de test : 
dico_data_test = {
    'features': features_test,
    'class': partis_test
}

In [ ]:
# Enregistrement des deux dicos dans des fichiers Parquet : 
df_train = pd.DataFrame(dico_data_train)
df_train.to_parquet('data_train_vectorized.parquet')
print("Les données train ont été enregistrées au format Parquet.")


df_test = pd.DataFrame(dico_data_test)
df_test.to_parquet('data_test_vectorized.parquet')
print("Les données test ont été enregistrées au format Parquet.")